In [ ]:
!pip install transformers
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.4 MB/s 
     |████████████████████████████████| 6.6 MB 46.9 MB/s 
     |████████████████████████████████| 596 kB 62.2 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 585 kB 6.7 MB/s 
     |████████████████████████████████| 419 kB 70.6 MB/s 
     |████████████████████████████████| 140 kB 77.6 MB/s 
     |████████████████████████████████| 1.1 MB 67.3 MB/s 
     |████████████████████████████████| 271 kB 78.1 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 144 kB 69.2 MB/s 


In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", eos_token="</s>")

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
import torch
from transformers import GPT2Config, GPT2LMHeadModel

pretrained_model_config = GPT2Config.from_pretrained("skt/kogpt2-base-v2")
model = GPT2LMHeadModel(pretrained_model_config)

fine_tuned_model_ckpt = torch.load("/content/drive/MyDrive/11_recipe_generator/model/epoch=3-val_loss=2.19-v1.ckpt", map_location=torch.device("cpu"))
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

<All keys matched successfully>

In [ ]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
def input_format(title, ingredients):
  """ transform user's input to fit model's input form
      Args:
        title: 요리 이름
        ingredients: 재료들 : list
      
      return:
        <unused0>title<unused1><unused2>재료1$재료2$재료3<unused3>
  """

  temp1 = '<unused0>' + title + '<unused1>'
  temp2 = '<unused2>'
  for ingredient in ingredients:
    temp2 += ingredient + '$'
  temp2 += '<unused3>'
  return temp1 + temp2


def inference(
    prompt, min_length=50, max_length=500, top_p=1.0, top_k=50, repetition_penalty=1.0, no_repeat_ngram_size=0, temperature=1.0):
  try:
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    with torch.no_grad():
      generated_ids = model.generate(
          input_ids,
          do_sample=True,
          top_p=float(top_p),
          top_k=int(top_k),
          min_length = int(min_length),
          max_length=int(max_length),
          repetition_penalty = float(repetition_penalty),
          no_repeat_ngram_size = int(no_repeat_ngram_size),
          temperature = float(temperature),
          pad_token_id=tokenizer.pad_token_id,
          eos_token_id=tokenizer.eos_token_id,
          bos_token_id=tokenizer.bos_token_id,
          use_cache=True
      )
      generated_sentence = tokenizer.decode(generated_ids[0])

  except:
    generated_sentence = "오류"

  return {
      'result': generated_sentence
  }

# 레시피 생성

In [ ]:
sentence = input_format("잣국수", ["잣", "소면", "소금"])
inference(sentence)

{'result': '<unused0> 잣국수<unused1><unused2> 잣$소면$소금$<unused3><unused4> 냄비에 물과 소금을 넣고 육수를 만든다. 육수가 끓기 시작하면 육수는 건진다. 건진 육수는 체에 걸러 차게 준비한다. 잣은 가로 세로 1.5 크기로 깍둑 썬다. 끓는 동안 소면을 끓이고 이 전 와 나머지 면 사리를 끓인다. 완성한다<unused5> \n</s>'}